In [37]:
%load_ext cython
!pip install line_profiler
%load_ext line_profiler

The cython extension is already loaded. To reload it, use:
  %reload_ext cython
The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


# **03-Python vs Cython vs NumPy - Speed Test**

## Vanilla Python

In [38]:
def clip_py(a, min_value, max_value):
    return min(max(a, min_value), max_value)


def compute_py(array_1, array_2, a, b, c):
    """
    This function must implement the formula
    np.clip(array_1, 2, 10) * a + array_2 * b + c

    array_1 and array_2 are 2D.
    """
    x_max = array_1.shape[0]
    y_max = array_1.shape[1]

    assert array_1.shape == array_2.shape

    result = np.zeros((x_max, y_max), dtype=array_1.dtype)

    for x in range(x_max):
        for y in range(y_max):
            tmp = clip_py(array_1[x, y], 2, 10)
            tmp = tmp * a + array_2[x, y] * b
            result[x, y] = tmp + c

    return result

## Avanced Cython

In [39]:
%%cython

import numpy as np
# We now need to fix a datatype for our arrays. I've used the variable
# DTYPE for this, which is assigned to the usual NumPy runtime
# type info object.
DTYPE = np.intc

# cdef means here that this function is a plain C function (so faster).
# To get all the benefits, we type the arguments and the return value.
cdef int clip_cy(int a, int min_value, int max_value):
    return min(max(a, min_value), max_value)

def compute_cy(int[:, :] array_1, int[:, :] array_2, int a, int b, int c):

    cdef Py_ssize_t x_max = array_1.shape[0]
    cdef Py_ssize_t y_max = array_1.shape[1]

    # array_1.shape is now a C array, no it's not possible
    # to compare it simply by using == without a for-loop.
    # To be able to compare it to array_2.shape easily,
    # we convert them both to Python tuples.
    assert tuple(array_1.shape) == tuple(array_2.shape)

    result = np.zeros((x_max, y_max), dtype=DTYPE)
    cdef int[:, :] result_view = result

    cdef int tmp
    cdef Py_ssize_t x, y

    for x in range(x_max):
        for y in range(y_max):

            tmp = clip_cy(array_1[x, y], 2, 10)
            tmp = tmp * a + array_2[x, y] * b
            result_view[x, y] = tmp + c

    return result

## Comparison Python-Cython-NumPy

In [40]:
array_1 = np.random.uniform(0, 500, size=(1500, 1000)).astype(np.intc)
array_2 = np.random.uniform(0, 500, size=(1500, 1000)).astype(np.intc)
a, b, c = 4, 3, 9

import time

def compute_np(array_1, array_2, a, b, c):
    return np.clip(array_1, 2, 10) * a + array_2 * b + c

In [41]:
%%timeit -n1 -r1
compute_py(array_1, array_2, a, b, c)

1 loop, best of 1: 11.5 s per loop


In [42]:
%%timeit -n10 -r2
compute_cy(array_1, array_2, a, b, c)

10 loops, best of 2: 7.06 ms per loop


In [43]:
%%timeit -n10 -r2
compute_np(array_1, array_2, a, b, c)

10 loops, best of 2: 4.9 ms per loop


# **05-Cython Cdef Example**

In [44]:
%%cython --a

cdef int a = 0
# cdef int i = 0
for i in range(10):
    a += i
print(a)

45


## Benchmark cdef

In [45]:
%%cython

def wocdef():
    a = 0
    i = 0
    for i in range(10000):
        a += i
    return a

In [46]:
%%cython

def wcdef():
    cdef int a = 0
    cdef int i = 0
    for i in range(10000):
        a += i
    return a

In [47]:
%%timeit -n10 -r1
wocdef()

10 loops, best of 1: 593 µs per loop


In [48]:
%%timeit -n10 -r1 
wcdef()

10 loops, best of 1: 366 ns per loop


## Function Declaration

In [49]:
def f_py(x):
    return x ** 2 - x


def integrate_f_py(a, b, N):
    s = 0
    dx = (b - a) / N
    for i in range(N):
        s += f_py(a + i * dx)
    return s * dx

In [50]:
%%cython --a

def f_cy(double x):
    return x ** 2 - x


def integrate_f_cy(double a,double b, int N):
    cdef int i = 0
    cdef double dx = (b - a) / N, s=0
    for i in range(N):
        s += f_cy(a + i * dx)
    return s * dx

In [51]:
%%timeit -n10 -r10
integrate_f_py(0.0, 100.0, 100000)

10 loops, best of 10: 20.9 ms per loop


In [52]:
%%timeit -n10 -r10
integrate_f_cy(0.0, 100.0, 100000)

10 loops, best of 10: 4.26 ms per loop


In [53]:
%%cython --a

cpdef double f_cy(double x):
    return x ** 2 - x


cpdef double integrate_f_cy(double a,double b, int N):
    cdef int i = 0
    cdef double dx = (b - a) / N, s=0
    for i in range(N):
        s += f_cy(a + i * dx)
    return s * dx

In [54]:
%%timeit -n10 -r10
integrate_f_cy(0.0, 100.0, 100000)

10 loops, best of 10: 159 µs per loop


## "except" clause

In [55]:
%%cython --a

cpdef double f_cy(double x) except? -1:
    return x ** 2 - x


cpdef double integrate_f_cy(double a,double b, int N):
    cdef int i = 0
    cdef double dx = (b - a) / N, s=0
    for i in range(N):
        s += f_cy(a + i * dx)
    return s * dx

In [56]:
%%timeit -n100 -r100
integrate_f_cy(0.0, 100.0, 100000)

100 loops, best of 100: 188 µs per loop


# **Profiling with line_profiler (only with Python)**

In [59]:
%lprun -f integrate_f_py integrate_f_py(0.0, 100.0, 100000)

## **Simple Typed Memoryview**

In [60]:
%%cython -a

def add_one(int[:,:] arr):
    for i in range(arr.shape[0]):
        for j in range(arr.shape[1]):
            arr[i,j] += 1

In [61]:
def add_one_py(arr):
    arr + np.ones(arr.shape)
#     for i in range(arr.shape[0]):
#         for j in range(arr.shape[1]):
#             arr[i,j] += 1

In [62]:
rar = np.random.randint(0,1000,size=(500,500),dtype=np.int32)

In [64]:
%%timeit -n100 -r5
add_one_py(rar)

100 loops, best of 5: 539 µs per loop


In [63]:
%%timeit -n100 -r5
add_one(rar)

100 loops, best of 5: 170 µs per loop


## **Different degrees of optimization**

In [65]:
def py_total(arr):
    I, i = arr.shape[0], 0
    J, j = arr.shape[1], 0
    K, k = arr.shape[2], 0
    total = 0
    for i in range(I):
        for j in range(J):
            for k in range(K):
                total += arr[i, j, k]
    return total

In [66]:
%%cython -a
def cy1_total(arr):
    cdef unsigned I = arr.shape[0], i=0
    cdef unsigned J = arr.shape[1], j=0
    cdef unsigned K = arr.shape[2], k=0
    cdef int total = 0
    for i in range(I):
        for j in range(J):
            for k in range(K):
                total += arr[i, j, k]
    return total

In [67]:
%%cython -a
def cy2_total(int[:,:,:] arr):
    cdef unsigned I = arr.shape[0], i=0
    cdef unsigned J = arr.shape[1], j=0
    cdef unsigned K = arr.shape[2], k=0
    cdef int total = 0
    for i in range(I):
        for j in range(J):
            for k in range(K):
                total += arr[i, j, k]
    return total

In [68]:
%%cython -a
cpdef int cy3_total(int[:,:,:] arr):
    cdef unsigned I = arr.shape[0], i=0
    cdef unsigned J = arr.shape[1], j=0
    cdef unsigned K = arr.shape[2], k=0
    cdef int total = 0
    for i in range(I):
        for j in range(J):
            for k in range(K):
                total += arr[i, j, k]
    return total

In [69]:
myarray = np.random.randint(0,1e2,size=(100,100,100),dtype=np.int32)

In [70]:
%%timeit -n5 -r1
py_total(myarray)

5 loops, best of 1: 402 ms per loop


In [74]:
%%timeit -n100 -r20
np.sum(myarray)

100 loops, best of 20: 1.05 ms per loop


In [71]:
%%timeit -n5 -r1
cy1_total(myarray)

5 loops, best of 1: 2.47 s per loop


In [72]:
%%timeit -n100 -r20
cy2_total(myarray)

100 loops, best of 20: 746 µs per loop


In [73]:
%%timeit -n100 -r20
cy3_total(myarray)

100 loops, best of 20: 749 µs per loop


# 07 Hands-On: Mandelbrot Fractale

In [76]:
def mandelbrot_cython(m, size, iterations):
    for i in range(size):
        for j in range(size):
            c = -2 + 3./size*j + 1j*(1.5-3./size*i)
            z = 0
            for n in range(iterations):
                if np.abs(z) <= 10:
                    z = z*z + c
                    m[i, j] = n
                else:
                    break

In [77]:
size = 400
iterations = 100
s = (size, size)
m = np.zeros(s, dtype=np.int32)

In [78]:
%%timeit -n1 -r1
mandelbrot_cython(m, size, iterations)

1 loop, best of 1: 7.44 s per loop
